# SKA-Mid conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.36 already installed.


## Download dataset

In [2]:
import graphviper

graphviper.utils.data.download(file="AA2-Mid-sim_00000.ms")

[2024-08-29 14:20:16,713]  WARNING  graphviper:  File exists: /home/fedemp/ws_xradio_pointing/venv_xradio_python_39/lib/python3.9/site-packages/graphviper/utils/data/.dropbox 
[2024-08-29 14:20:16,713]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List         
 ────────────────────── 
  AA2-Mid-sim_00000.ms

AA2-Mid-sim_00000.ms.zip:   0%|          | 0.00/14.0M [00:00<?, ?iB/s]

## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "AA2-Mid-sim_00000.ms"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "AA2-Mid-sim_00000.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-08-29 14:20:23,075]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-29 14:20:23,080]     INFO  graphviper:  Number of partitions: 1 
[2024-08-29 14:20:23,081]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [0] 
[2024-08-29 14:20:23,358]  WARNING  graphviper:  SOURCE table empty for (unique) source_id [0] and spectral_window_id 0. 


## Processing Set

In [4]:
ps_name = outfile

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,AA2-Mid-sim_00000_0,[obs_0],"(95, 1891, 3, 4)","[XX, XY, YX, YY]",[0],spw_0,[_0],[Unknown],[],"[fk5, 0h00m00.00s, -45d00m00.00s]",950000000.0,9.500989e+08


In [5]:
ps["AA2-Mid-sim_00000_0"].attrs["antenna_xds"]

<xarray.Dataset> Size: 11kB
Dimensions:                (antenna_name: 62, cartesian_pos_label: 3,
                            receptor_label: 2, sky_dir_label: 2)
Coordinates:
  * antenna_name           (antenna_name) <U5 1kB 's0000' 's0001' ... 's0061'
  * cartesian_pos_label    (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                  (antenna_name) <U5 1kB dask.array<chunksize=(62,), meta=np.ndarray>
    polarization_type      (antenna_name, receptor_label) <U1 496B dask.array<chunksize=(62, 2), meta=np.ndarray>
  * receptor_label         (receptor_label) <U5 40B 'pol_0' 'pol_1'
  * sky_dir_label          (sky_dir_label) <U3 24B 'ra' 'dec'
    station                (antenna_name) <U5 1kB dask.array<chunksize=(62,), meta=np.ndarray>
Data variables:
    ANTENNA_DISH_DIAMETER  (antenna_name) float64 496B dask.array<chunksize=(62,), meta=np.ndarray>
    ANTENNA_FEED_OFFSET    (antenna_name, cartesian_pos_label) float64 1kB dask.array<chunksize=(62, 3), meta=np.ndarray>
    ANTENNA_POSITION       (antenna_name, cartesian_pos_label) float64 1kB dask.array<chunksize=(62, 3), meta=np.ndarray>
    BEAM_OFFSET            (antenna_name, receptor_label, sky_dir_label) float64 2kB dask.array<chunksize=(62, 2, 2), meta=np.ndarray>
    RECEPTOR_ANGLE         (antenna_name, receptor_label) float64 992B dask.array<chunksize=(62, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  OSKAR 2.8.4-dev 2023-08-22 d1b86f8c
    relocatable_antennas:    False
    type:                    antenna